In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import calendar
import plotly
plotly.offline.init_notebook_mode()
from plotly.subplots import make_subplots
from IPython.display import Markdown, display
import atoti as tt

In [ ]:
df = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\GFK data\202206\GFK_CUSTOMREPORT_TRUCKTIRE_TW_BRIDGESTONE_Jun22.csv")

In [ ]:
def gfk_preprocessing(path):
    ## 讀取gfk原始檔
    df = pd.read_csv(path)
    ## 格式化日期(Period)和第一次上市日期(FirstActivity)
    df["Period"] = pd.to_datetime(df["Period"], format="%b%y")
    df["FirstActivity"] = pd.to_datetime(df["FirstActivity"], format="%B %Y")
    ## 去除BRAND裡面的UNBRANDED
    df["BRAND"] = df.apply(lambda x: x["Item"].split(" ")[-1] if x["BRAND"] == "UNBRANDED" else x["BRAND"], axis=1)
    ## 需要的日期時間，使用Period最大的日期，也就是最新的報告，然後需要看到過去14個月的數據
    time_condition = df["Period"].max() - relativedelta(months=14)
    filtered_df = df[df["Period"] >= time_condition]
    # 將全部的numeric都改成int
    filtered_df = filtered_df.astype({"SALES UNITS": int, "SALES <LC>": int, "PRICE TWD/UN.": int})
    return filtered_df

In [ ]:
gfk = gfk_preprocessing(r"D:\kc.hsu\OneDrive - Bridgestone\GFK data\202206\GFK_CUSTOMREPORT_TRUCKTIRE_TW_BRIDGESTONE_Jun22.csv")

In [ ]:
gfk.columns = gfk.columns.str.replace("'"," ").str.replace("/", "-")

In [ ]:
result = gfk[['Period', 'Item', 'BRAND', 'REGION', 'BRIDGESTONE TYPE',
       'DIMENSION (Truck Tyres)', 'RIMDIAMET. -MM', 'FirstActivity', 'SALES UNITS', 'SALES <LC>',
       'PRICE TWD-UN.']]

In [ ]:
result

In [ ]:
session = tt.Session()

In [ ]:
result.columns

In [ ]:
gfk_table = session.read_pandas(result, table_name="gfk", keys=["Period", "Item", "BRAND", "REGION", "DIMENSION (Truck Tyres)"])

In [ ]:
cube = session.create_cube(gfk_table)

In [ ]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [ ]:
cube.schema

In [ ]:
session.link()

In [ ]:
class gfk_report:
    
    def __init__(self, path, months):
        ## 讀取gfk原始檔
        self.df = pd.read_csv(path)
        self.months = months - 1
        ## 格式化日期(Period)和第一次上市日期(FirstActivity)
        self.df["Period"] = pd.to_datetime(self.df["Period"], format="%b%y")
        self.df["FirstActivity"] = pd.to_datetime(self.df["FirstActivity"], format="%B %Y")
        ## 去除BRAND裡面的UNBRANDED
        self.df["BRAND"] = self.df.apply(lambda x: x["Item"].split(" ")[-1] if x["BRAND"] == "UNBRANDED" else x["BRAND"], axis=1)
        ## 需要的日期時間，使用Period最大的日期，也就是最新的報告，然後需要看到過去14個月的數據
        # time_condition = self.df["Period"].max() - relativedelta(months=14)
        # filtered_df = self.df[self.df["Period"] >= time_condition]
        # 將全部的numeric都改成int
        filtered_df = self.df.astype({"SALES UNITS": int, "SALES <LC>": int, "PRICE TWD/UN.": int})
        self.df = filtered_df
        
    def gfk_preprocessing(self):
        self.cur_month = self.df["Period"].max().to_pydatetime().strftime("%Y-%m-%d")
        self.last_month = (self.df["Period"].max() - relativedelta(months=1)).to_pydatetime().strftime("%Y-%m-%d")
        self.last_year_month = (self.df["Period"].max() - relativedelta(months=12)).to_pydatetime().strftime("%Y-%m-%d")
        self.begin_year = (self.df["Period"].max() - relativedelta(months=self.months)).to_pydatetime().strftime("%Y-%m-%d")
        self.begin_last_year = (self.df["Period"].max() - relativedelta(months=(12 + self.months))).to_pydatetime().strftime("%Y-%m-%d")
        self.cur_month_data = self.df[self.df["Period"] == self.cur_month]
        self.last_month_data = self.df[self.df["Period"] == self.last_month]
        self.mom_data = self.df[self.df["Period"] == self.last_year_month]
        self.ytd_this_year = self.df[self.df["Period"].between(self.begin_year, self.cur_month)]
        self.ytd_last_year = self.df[self.df["Period"].between(self.begin_last_year, self.last_year_month)]
        
    def tbr_g_demand(self):
        cur_month_volume = self.cur_month_data["SALES UNITS"].sum()
        last_month_volume = self.last_month_data["SALES UNITS"].sum()
        last_year_month_volume = self.mom_data["SALES UNITS"].sum()
        ytd_this_year_volumne = self.ytd_this_year["SALES UNITS"].sum()
        ytd_last_year_volumne = self.ytd_last_year["SALES UNITS"].sum()
        last_month_name = calendar.month_name[self.months]
        current_month_name = calendar.month_name[self.months + 1]
        cur_vs_last = "{:.0f}%".format((cur_month_volume / last_month_volume) * 100)
        cur_vs_last_year = "{:.0f}%".format((cur_month_volume / last_year_month_volume) * 100)
        ytd_ratio = "{:.0f}%".format((ytd_this_year_volumne / ytd_last_year_volumne) * 100)
        # print("TBR(G)\nDemand\na. {} vs {} {}, vs PY {}\nb. YTD vs PY {}".format(current_month_name, last_month_name, cur_vs_last, cur_vs_last_year, ytd_ratio))
        display(Markdown(f"## TBR(G)"))
        display(Markdown(f"### Demand"))
        display(Markdown(f"a. {current_month_name} vs {last_month_name} **{cur_vs_last}**, vs PY **{cur_vs_last_year}**"))
        display(Markdown(f"b. YTD vs PY **{ytd_ratio}**"))
        
    def tbr_g_demand_trend_plot(self):
        general_table = self.df.groupby([pd.Grouper(key="Period", freq="m"), "BRIDGESTONE TYPE"])[["SALES UNITS"]].sum().reset_index()
        general_table["%"] = 100 * (general_table["SALES UNITS"] / general_table.groupby(["Period"])["SALES UNITS"].transform("sum"))
        general_table["Total_month"] = general_table.groupby(["Period"])["SALES UNITS"].transform("sum")
        fig = px.bar(general_table, x="Period", y="SALES UNITS", color="BRIDGESTONE TYPE", barmode="stack", text=general_table["%"].map(lambda x: "{:.0f}%".format(x)), category_orders={"BRIDGESTONE TYPE": ["LSR2", "TBR", "EXCLUDED TBR AND LSR2"]}, hover_data=["Total_month"], title="TBR(G) Sales Units").update_layout(hoverlabel=dict(bgcolor="white", font_size=12, font_family="Calibre"), hovermode="x")
        fig.show()
        
    def tbr_g_demand_ytd_plot(self):
        quar_fig = self.df.groupby([pd.Grouper(key="Period", freq="QS"), "BRIDGESTONE TYPE"])[["SALES UNITS"]].sum().reset_index()
        quar_fig["%"] = 100 * (quar_fig["SALES UNITS"] / quar_fig.groupby(["Period"])["SALES UNITS"].transform("sum"))
        quar_fig["Total_month"] = quar_fig.groupby(["Period"])["SALES UNITS"].transform("sum")
        quar_fig = px.bar(quar_fig, x="Period", y="SALES UNITS", color="BRIDGESTONE TYPE", barmode="stack", text=quar_fig["%"].map(lambda x: "{:.0f}%".format(x)), category_orders={"BRIDGESTONE TYPE": ["LSR2", "TBR", "EXCLUDED TBR AND LSR2"]}, hover_data=["Total_month"], title="YTD TBR(G) Sales Units").update_layout(hoverlabel=dict(bgcolor="white", font_size=12, font_family="Calibre"), hovermode="x")
        quar_fig.show()
        

    

In [ ]:
test = gfk_report("/Users/kai/Library/CloudStorage/OneDrive-Bridgestone/GFK data/202206/GFK_CUSTOMREPORT_TRUCKTIRE_TW_BRIDGESTONE_Jun22.csv", 6)

In [ ]:
test.gfk_preprocessing()

In [ ]:
test.tbr_g_demand()

In [ ]:
test.tbr_g_demand_trend_plot()

In [ ]:
test.tbr_g_demand_ytd_plot()

## 開始作圖
兩個方向
- **Price的部分**
    1. 品牌只需要`普利司通，瑪吉斯，米其林，玲瓏，其他`
- **Share的部分**
    1. 品牌需要`普利司通，瑪吉斯，米其林，玲瓏，JINYU，PRESA，DUNLOP，GOODYEAR，YOKOHAMA，其他`

In [ ]:
top_10_list = gfk.groupby([pd.Grouper(key="Period", freq="y"), "BRAND"])[["SALES UNITS"]].sum().reset_index().sort_values(["Period", "SALES UNITS"], ascending=False)["BRAND"].head(9).tolist()
top_5_list = gfk.groupby([pd.Grouper(key="Period", freq="y"), "BRAND"])[["SALES UNITS"]].sum().reset_index().sort_values(["Period", "SALES UNITS"], ascending=False)["BRAND"].head(4).tolist()

In [ ]:
gfk.columns

In [ ]:
general_table = gfk.groupby([pd.Grouper(key="Period", freq="m"), "BRIDGESTONE TYPE"])[["SALES UNITS"]].sum().reset_index()

In [ ]:
general_table["%"] = 100 * (general_table["SALES UNITS"] / general_table.groupby(["Period"])["SALES UNITS"].transform("sum"))

In [ ]:
general_table["Total_month"] = general_table.groupby(["Period"])["SALES UNITS"].transform("sum")

In [ ]:
general_table.columns

In [ ]:
fig = px.bar(general_table, x="Period", y="SALES UNITS", color="BRIDGESTONE TYPE", barmode="stack", text=general_table["%"].map(lambda x: "{:.0f}%".format(x)), category_orders={"BRIDGESTONE TYPE": ["LSR2", "TBR", "EXCLUDED TBR AND LSR2"]}, hover_data=["Total_month"], width=720).update_layout(hoverlabel=dict(bgcolor="white", font_size=12, font_family="Calibre"), hovermode="x")

In [ ]:
fig.show()

In [ ]:
colors = ["yellow", "orange", "blue"]
category = ["TBR", "LSR2", "EXCLUDED TBR AND LSR2"]
category_colors = dict(zip(category, colors))

In [ ]:
general_table["color"] = general_table["BRIDGESTONE TYPE"].map(category_colors)

In [ ]:
fig = go.Figure()
for brand, value in general_table.groupby(["BRIDGESTONE TYPE", "color"]):
    fig.add_trace(go.Bar(x=value["Period"], y=value["SALES UNITS"], name=brand[0], text=value["%"].map(lambda x: "{:.0f}%".format(x)), textposition="inside", legendgroup="group1", marker=dict(color=brand[1])))
fig.update_layout(barmode="stack")
fig.show()

In [ ]:
quar_fig = gfk.groupby([pd.Grouper(key="Period", freq="QS"), "BRIDGESTONE TYPE"])[["SALES UNITS"]].sum().reset_index()

In [ ]:
quar_fig["%"] = 100 * (quar_fig["SALES UNITS"] / quar_fig.groupby(["Period"])["SALES UNITS"].transform("sum"))
quar_fig["Total_month"] = quar_fig.groupby(["Period"])["SALES UNITS"].transform("sum")

In [ ]:
quar_fig["color"] = quar_fig["BRIDGESTONE TYPE"].map(category_colors)

In [ ]:
quar_fig = px.bar(quar_fig, x="Period", y="SALES UNITS", color="BRIDGESTONE TYPE", barmode="stack", text=quar_fig["%"].map(lambda x: "{:.0f}%".format(x)), category_orders={"BRIDGESTONE TYPE": ["LSR2", "TBR", "EXCLUDED TBR AND LSR2"]}, hover_data=["Total_month"], width=720).update_layout(hoverlabel=dict(bgcolor="white", font_size=12, font_family="Calibre"), hovermode="x", showlegend=False)

In [ ]:
quar_fig = px.bar(quar_fig, x="Period", y="SALES UNITS", color="BRIDGESTONE TYPE", barmode="stack", text=quar_fig["%"].map(lambda x: "{:.0f}%".format(x)), category_orders={"BRIDGESTONE TYPE": ["LSR2", "TBR", "EXCLUDED TBR AND LSR2"]}, hover_data=["Total_month"], width=720)

In [ ]:
quar = go.Figure()
for brand, value in quar_fig.groupby(["BRIDGESTONE TYPE", "color"]):
    quar.add_trace(go.Bar(x=value["Period"], y=value["SALES UNITS"], name=brand[0], text=value["%"].map(lambda x: "{:.0f}%".format(x)), textposition="inside", legendgroup="group", showlegend=True, marker=dict(color=brand[1])))
quar.update_layout(barmode="stack")
quar.show()

In [ ]:
tes_p = make_subplots(rows=1, cols=2, specs=[[{"type":"bar"}, {"type": "bar"}]])

In [ ]:
t1 = []
t2 = []
for t in range(len(fig["data"])):
    t1.append(fig["data"][t])
for x in range(len(quar["data"])):
    t2.append(quar["data"][x])

In [ ]:
for traces in t1:
    tes_p.append_trace(traces, row=1, col=1)
for traces in t2:
    tes_p.append_trace(traces, row=1, col=2)
tes_p.update_layout(legend_title_text = "Brand", barmode="stack", title="TBR(G) Market Share in Units",
                   hoverlabel=dict(bgcolor="white", font_size=12, font_family="Calibre"), hovermode="x", 
                   legend=dict(groupclick="toggleitem"))

tes_p.show()

In [ ]:
## go.Figure來做圖

In [ ]:
quar = gfk.groupby([pd.Grouper(key="Period", freq="QS"), "BRAND"])[["SALES UNITS"]].sum().reset_index()

In [ ]:
quar["BRAND"] = quar["BRAND"].map(lambda x: "Others" if x not in top_5_list else x)

In [ ]:
quar["SALES UNITS in %"] = 100 * quar["SALES UNITS"] / quar.groupby("Period")["SALES UNITS"].transform("sum")

In [ ]:
quar = quar.groupby(["Period", "BRAND"])[["SALES UNITS", "SALES UNITS in %"]].sum().reset_index()

In [ ]:
quar

In [ ]:
quar_fig = go.Figure()
for brand, value in quar.groupby("BRAND"):
    quar_fig.add_trace(go.Bar(x=value["Period"], y=value["SALES UNITS"], name=brand, text=value["SALES UNITS in %"].map(lambda x: "{:.1f}%".format(x)), textposition="inside", legendgroup="group", showlegend=False))
quar_fig.update_layout(legend_title_text = "Brand", barmode="stack", title="TBR(G) Market Share in Units", hoverlabel=dict(bgcolor="white", font_size=12, font_family="Calibre"), hovermode="x")
quar_fig.show()

In [ ]:
general_table